In [10]:
# Lab: Playing and Winning the CHSH Game with Qiskit!
# Based on the Presentation: "Unlock Quantum Spookiness: Beating the Odds with Entanglement!"

# Welcome to the CHSH Game Lab!
# In this lab, you will implement the quantum strategy for the CHSH game in Qiskit.
# You will simulate the game, observe the power of entanglement, and verify that quantum mechanics
# allows Alice and Bob to win more often than is possible with any classical strategy.

# Let's get started!

# Part I: Introduction to Qiskit and the CHSH Game

# Before starting with the exercises, please run the cell below by pressing 'shift' + 'return'.
# You can run the other following cells in the same way.

#!pip install qiskit # Uncomment if you need to install Qiskit
import qiskit
import numpy as np
from numpy import pi

# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, transpile, assemble
from qiskit.quantum_info import Statevector, random_statevector
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex

# Additional required imports:
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.circuit.library import HGate, CXGate, RYGate, ZGate, XGate, SGate, SdgGate
from qiskit_aer import AerSimulator

qiskit.__version__
print("Setup complete! Qiskit libraries imported and simulator backend ready.")

Setup complete! Qiskit libraries imported and simulator backend ready.


## Introduction to the CHSH Game

In this lab, we will explore the fascinating CHSH game and demonstrate how quantum entanglement can provide a winning strategy that outperforms any classical approach.

**Recall the CHSH Game:**

Imagine a game with two players, Alice and Bob, who are spatially separated and cannot communicate once the game begins. A referee provides Alice with a random question 'X' (either 0 or 1) and Bob with a random question 'Y' (either 0 or 1). Based on their questions, Alice must produce an answer 'A' (either 0 or 1) and Bob must produce an answer 'B' (either 0 or 1).

They win or lose based on the following rules, which depend on the questions X and Y:

*   **If X=0 and Y=0, or X=0 and Y=1, or X=1 and Y=0:** They win if their answers are the same (A = B).
*   **If X=1 and Y=1:** They win if their answers are different (A ≠ B).

**Classical Limit:**

It can be mathematically proven that with any classical strategy, Alice and Bob can win in the CHSH game at most 75% of the time on average. This is known as the classical limit.

**Quantum Advantage:**

However, using the principles of quantum mechanics, specifically entanglement, Alice and Bob can devise a strategy to win approximately 85% of the time, thus surpassing the classical limit. This lab will guide you to implement and verify this quantum strategy using Qiskit!

Let's start by defining the building blocks for our quantum strategy in Qiskit.

## Step 1: Define the U_theta Gate in Qiskit

In the quantum CHSH strategy, both Alice and Bob use a rotation gate, denoted as U(θ).  Let's create a function in Qiskit to represent this gate.  We'll use the `RYGate` which performs a rotation around the Y-axis.

**Task 1:** Run the code cell below to define the `u_theta(theta)` function.

*Take a moment to understand the code.*  This function takes an angle `theta` as input and returns a Qiskit `Instruction` representing the U(θ) gate. We convert it to an instruction so we can easily append it to our quantum circuits later.

In [11]:
def u_theta(theta):
    """Creates the U(theta) gate as a Qiskit QuantumCircuit."""
    qc = QuantumCircuit(1, name=f'U({theta:.2f})') # 1 qubit circuit, named for theta value
    qc.ry(theta, 0) # Apply RY rotation to qubit 0
    return qc.to_instruction() # Convert to gate instruction for circuit use

print("u_theta(theta) function defined!")

u_theta(theta) function defined!


## Step 2: Build the Quantum CHSH Circuit Function

Now, we'll create the core function that implements Alice and Bob's quantum strategy for the CHSH game. This function, `quantum_chsh_circuit(x, y)`, will take the referee's questions (x and y) as input and construct the quantum circuit according to the strategy we discussed in the presentation.

**Task 2:** Run the code cell below to define the `quantum_chsh_circuit(x, y)` function.

*Carefully examine the code.*  It follows the 4-step quantum strategy:

1.  **Bell State Preparation:** It starts by creating the Bell state Φ+ using a Hadamard gate and a CNOT gate.
2.  **Alice's Rotation:** Based on input `x`, it applies the `u_theta` gate with angle 0 (if x=0) or π/2 (if x=1) to qubit 0 (Alice's qubit).
3.  **Bob's Rotation:** Based on input `y`, it applies the `u_theta` gate with angle π/8 (if y=0) or -π/8 (if y=1) to qubit 1 (Bob's qubit).
4.  **Measurement:** Finally, it measures both qubits and stores the outcomes in classical bits.

In [12]:
def quantum_chsh_circuit(x, y):
    """Creates the quantum circuit for Alice and Bob's strategy given inputs x and y."""
    circuit = QuantumCircuit(2, 2) # 2 qubits, 2 classical bits

    # Prepare Bell State (Phi+)
    circuit.h(0)
    circuit.cx(0, 1)

    # Alice's rotation based on x
    alice_theta = 0 if x == 0 else np.pi/2 # 0 for x=0, pi/2 for x=1
    circuit.append(u_theta(alice_theta), [0]) # Apply U(alice_theta) to qubit 0

    # Bob's rotation based on y
    bob_theta = np.pi/8 if y == 0 else -np.pi/8 # pi/8 for y=0, -pi/8 for y=1
    circuit.append(u_theta(bob_theta), [1]) # Apply U(bob_theta) to qubit 1

    circuit.measure([0, 1], [0, 1]) # Measure both qubits
    return circuit

print("quantum_chsh_circuit(x, y) function defined!")

quantum_chsh_circuit(x, y) function defined!


## Step 3: Simulate the CHSH Game and Verify Winning Probabilities

Now for the exciting part! We will simulate the CHSH game using our `quantum_chsh_circuit` function for all four possible input combinations (XY = 00, 01, 10, 11).  For each combination, we'll run the simulation many times (`shots=1024`) and analyze the results to calculate the winning probability.

**Task 3:** Run the code cell below to simulate the CHSH game and calculate winning probabilities.

*Observe the output carefully.* The code will:

*   Loop through all four input combinations (x, y).
*   For each combination:
    *   Create the quantum circuit using `quantum_chsh_circuit(x, y)`.
    *   Run the simulation using the `qasm_simulator`.
    *   Get the measurement counts.
    *   Determine the number of wins based on the CHSH game winning conditions (remembering to win when A=B for XY=00, 01, 10 and when A!=B for XY=11).
    *   Calculate the winning probability.
    *   Print the winning probability for that input combination.
*   Finally, it will calculate and print the average winning probability across all four input cases.

*After running the code, check if the winning probabilities for each case and the average winning probability are approximately **85%** (or 0.853...). This is the key result we expect from the quantum strategy!*

In [13]:
simulator = AerSimulator() # Using AerSimulator for potentially better performance
shots = 1024 # Number of game rounds to simulate

winning_probabilities = {} # Dictionary to store winning probabilities for each input

for x in [0, 1]:
    for y in [0, 1]:
        circuit = quantum_chsh_circuit(x, y)
        job = simulator.run(transpile(circuit, simulator), shots=shots) # Transpile for efficiency
        result = job.result()
        counts = result.get_counts(circuit)

        wins = 0
        for outcome, count in counts.items():
            alice_outcome = int(outcome[1]) # Alice's outcome is the second bit in the outcome string
            bob_outcome = int(outcome[0])   # Bob's outcome is the first bit in the outcome string

            # Determine win condition based on x and y
            if (x == 1 and y == 1): # Case XY=11: Win if A != B
                if alice_outcome != bob_outcome:
                    wins += count
            else: # Cases XY=00, 01, 10: Win if A == B
                if alice_outcome == bob_outcome:
                    wins += count

        win_probability = wins / shots
        winning_probabilities[(x, y)] = win_probability
        print(f"Input (x={x}, y={y}): Winning Probability = {win_probability:.4f}")

print("\nWinning Probabilities for all Inputs:", winning_probabilities)
average_winning_probability = np.mean(list(winning_probabilities.values()))
print(f"Average Winning Probability (Quantum Strategy): {average_winning_probability:.4f}")

Input (x=0, y=0): Winning Probability = 0.9688
Input (x=0, y=1): Winning Probability = 0.9531
Input (x=1, y=0): Winning Probability = 0.7002
Input (x=1, y=1): Winning Probability = 0.7197

Winning Probabilities for all Inputs: {(0, 0): 0.96875, (0, 1): 0.953125, (1, 0): 0.7001953125, (1, 1): 0.7197265625}
Average Winning Probability (Quantum Strategy): 0.8354


### Visualize the Measurement Outcomes (Optional)

To get a better visual understanding of the outcomes, you can plot histograms for each input combination.

**Task (Optional):** Run the code cell below to generate histograms for each input case.

These histograms will show the distribution of measurement outcomes ('00', '01', '10', '11') for each input (x, y).  Observe how the probabilities are distributed.  For example, for cases where they win if A=B, you should see higher probabilities for '00' and '11' outcomes.

In [14]:
import matplotlib.pyplot as plt

for x in [0, 1]:
    for y in [0, 1]:
        circuit = quantum_chsh_circuit(x, y)
        job = simulator.run(transpile(circuit, simulator), shots=shots) # Transpile for efficiency
        result = job.result()
        counts = result.get_counts(circuit)

        plt.figure() # Create a new figure for each histogram
        plot_histogram(counts, title=f"Measurement Outcomes for Input (x={x}, y={y})")
        plt.show()


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.2.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\izzye\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\izzye\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "C:\Users\izzye\AppData\Roaming\Python\Python310\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\izzye\AppData\Roaming\Python\Python310\site-packages\traitlets\config\applicatio

AttributeError: _ARRAY_API not found

ImportError: numpy.core.multiarray failed to import

## Step 4: Verify Quantum Advantage - Beating the Classical Limit

**Question:**  Look at the average winning probability you calculated. Is it greater than 75% (0.75)?

**Answer:**  Yes! The average winning probability using the quantum strategy should be approximately **85%**, which is significantly higher than the classical limit of 75%.

**Congratulations!** You have successfully simulated the quantum CHSH game strategy and verified that quantum mechanics allows Alice and Bob to outperform any classical strategy!  This demonstrates the power of entanglement in achieving a "quantum advantage" in this game.

## Optional Challenges - Explore Further!

If you want to delve deeper and experiment more, try these optional challenges:

### Challenge 1: Simulate a Classical Strategy

To truly appreciate the quantum advantage, let's simulate a simple **classical strategy** and compare its performance to the quantum strategy.

**Example Classical Strategy:**  Alice and Bob decide on a strategy beforehand:

*   Alice always answers '0', regardless of the question X.
*   Bob always answers '0', regardless of the question Y.

**Task (Challenge 1):** Modify the code below to simulate this classical "Always 0" strategy for the CHSH game.

You will need to create a new function, for example, `classical_chsh_strategy(x, y)`, that *does not* use quantum circuits.  Instead, it should directly return Alice's answer (A=0) and Bob's answer (B=0) for any inputs x and y. Then, simulate the CHSH game using this classical strategy and calculate its average winning probability.

*Hint:* You won't need to use Qiskit for the classical strategy simulation. You can directly calculate wins and losses based on the rules and the fixed answers A=0 and B=0.

*After implementing the classical strategy simulation, compare its average winning probability to the ~85% you obtained with the quantum strategy.  You should see that the classical strategy's winning probability is indeed limited to 75% or less.*

In [ ]:
# Challenge 1 Code (Example - you need to complete the classical_chsh_strategy function)

def classical_chsh_strategy(x, y):
    """Implements the classical 'Always 0' strategy."""
    alice_answer = 0
    bob_answer = 0
    return alice_answer, bob_answer

classical_winning_probabilities = {}

for x in [0, 1]:
    for y in [0, 1]:
        alice_answer, bob_answer = classical_chsh_strategy(x, y)
        wins = 0 # Initialize wins for this input combination
        # Determine if they win based on classical answers and CHSH rules
        if (x == 1 and y == 1): # Case XY=11: Win if A != B
            if alice_answer != bob_answer:
                wins = 0 # They lose for this single "shot" as strategy is deterministic
        else: # Cases XY=00, 01, 10: Win if A == B
            if alice_answer == bob_answer:
                wins = 1 # They win for this single "shot"

        win_probability = wins # Probability is 1 if win, 0 if lose for deterministic strategy
        classical_winning_probabilities[(x, y)] = win_probability
        print(f"Classical Strategy - Input (x={x}, y={y}): Winning Probability = {win_probability:.4f}")


average_classical_winning_probability = np.mean(list(classical_winning_probabilities.values()))
print(f"Average Winning Probability (Classical 'Always 0' Strategy): {average_classical_winning_probability:.4f}")

print("\nComparing Quantum vs. Classical:")
print(f"Average Winning Probability (Quantum): {average_winning_probability:.4f}")
print(f"Average Winning Probability (Classical): {average_classical_winning_probability:.4f}")

# Compare average_classical_winning_probability with average_winning_probability (quantum)

### Challenge 2: Experiment with Angles

In the quantum strategy, Alice and Bob use specific rotation angles (0, π/2, π/8, -π/8).  What happens if you change these angles?

**Task (Challenge 2):** Modify the `quantum_chsh_circuit(x, y)` function to experiment with different rotation angles for Alice and Bob.

*Try changing the `alice_theta` and `bob_theta` values in the function.* For example:

*   Try setting all angles to 0.
*   Try using random angles.
*   Try slightly perturbing the original angles (e.g., π/8 + 0.1, -π/8 - 0.2).

*After changing the angles, re-run the simulation code from Step 3 and observe how the winning probabilities change.*

**Questions to consider:**

*   Do the winning probabilities increase or decrease when you change the angles?
*   Can you find any angles that improve the winning probability beyond ~85%? (Hint: You likely won't be able to, due to the Tsirelson bound).
*   What happens if you use angles that are completely different from the optimal ones?

In [ ]:
# Challenge 2 Code (Example - modify angles in this function and re-run simulations)

def quantum_chsh_circuit_modified_angles(x, y):
    """Quantum CHSH circuit with MODIFIED ROTATION ANGLES - EXPERIMENT HERE!"""
    circuit = QuantumCircuit(2, 2)

    circuit.h(0)
    circuit.cx(0, 1)

    # --- MODIFY ANGLES BELOW ---
    alice_theta = 0 if x == 0 else np.pi/4  # Example: Changed Alice's angle
    bob_theta = np.pi/16 if y == 0 else -np.pi/16  # Example: Changed Bob's angle
    # --- MODIFY ANGLES ABOVE ---

    circuit.append(u_theta(alice_theta), [0])
    circuit.append(u_theta(bob_theta), [1])

    circuit.measure([0, 1], [0, 1])
    return circuit


# Re-run simulation with the modified angle circuit (replace quantum_chsh_circuit with quantum_chsh_circuit_modified_angles in Step 3 code)
# --- Re-run Step 3 Simulation Code Here, but replace quantum_chsh_circuit with quantum_chsh_circuit_modified_angles ---
simulator = AerSimulator() # Using AerSimulator for potentially better performance
shots = 1024 # Number of game rounds to simulate

winning_probabilities_modified_angles = {} # Dictionary to store winning probabilities for each input

for x in [0, 1]:
    for y in [0, 1]:
        circuit = quantum_chsh_circuit_modified_angles(x, y) # <---- USE MODIFIED CIRCUIT HERE
        job = simulator.run(transpile(circuit, simulator), shots=shots) # Transpile for efficiency
        result = job.result()
        counts = result.get_counts(circuit)

        wins = 0
        for outcome, count in counts.items():
            alice_outcome = int(outcome[1]) # Alice's outcome is the second bit in the outcome string
            bob_outcome = int(outcome[0])   # Bob's outcome is the first bit in the outcome string

            # Determine win condition based on x and y
            if (x == 1 and y == 1): # Case XY=11: Win if A != B
                if alice_outcome != bob_outcome:
                    wins += count
            else: # Cases XY=00, 01, 10: Win if A == B
                if alice_outcome == bob_outcome:
                    wins += count

        win_probability = wins / shots
        winning_probabilities_modified_angles[(x, y)] = win_probability
        print(f"Modified Angles - Input (x={x}, y={y}): Winning Probability = {win_probability:.4f}")

print("\nWinning Probabilities for all Inputs (Modified Angles):", winning_probabilities_modified_angles)
average_winning_probability_modified_angles = np.mean(list(winning_probabilities_modified_angles.values()))
print(f"Average Winning Probability (Quantum Strategy - Modified Angles): {average_winning_probability_modified_angles:.4f}")

### Challenge 3: Explore Different Bell States (Advanced)

In this lab, we used the Bell state Φ+ to implement the quantum CHSH strategy.  There are other Bell states (Φ-, Ψ+, Ψ-).  Could using a different Bell state affect the winning probability or require changes to Alice and Bob's rotation angles?

**Task (Challenge 3 - Advanced):**  Modify the `quantum_chsh_circuit` function to use a different Bell state, for example, Ψ- = (|01> - |10>) / √2.

*Hint:*  You will need to change the Bell state preparation part of the circuit in the `quantum_chsh_circuit` function.  Research or recall how to create different Bell states using Qiskit gates (Hadamard, CNOT, X, Z).  For Ψ- you can add an X gate on qubit 1 *before* the CNOT and Hadamard gates.

*After modifying the circuit to use a different Bell state, re-run the simulation code and analyze the winning probabilities. You might also need to adjust Alice and Bob's rotation angles to optimize the strategy for a different Bell state.*

**Questions to consider:**

*   Does using a different Bell state significantly change the winning probability with the original angles?
*   Do you need to adjust Alice and Bob's rotation angles to achieve a high winning probability with a different Bell state?
*   Can you find a combination of a different Bell state and adjusted angles that performs better than the original strategy with Φ+? (Again, you likely won't exceed the Tsirelson bound).

In [ ]:
# Challenge 3 Code (Example - modify Bell state preparation and potentially angles)

def quantum_chsh_circuit_psi_minus(x, y):
    """Quantum CHSH circuit using Bell state PSI- and potentially modified angles."""
    circuit = QuantumCircuit(2, 2)

    # --- MODIFY BELL STATE PREPARATION BELOW to create PSI- ---
    circuit.x(1) # Create Psi- Bell State: Apply X to qubit 1 first
    circuit.h(0)
    circuit.cx(0, 1)
    # --- MODIFY BELL STATE PREPARATION ABOVE ---


    # --- You might also need to ADJUST ANGLES for PSI- if needed ---
    alice_theta = 0 if x == 0 else np.pi/2  # Original angles - might need adjustment for Psi-
    bob_theta = np.pi/8 if y == 0 else -np.pi/8  # Original angles - might need adjustment for Psi-
    # --- ADJUST ANGLES ABOVE ---

    circuit.append(u_theta(alice_theta), [0])
    circuit.append(u_theta(bob_theta), [1])

    circuit.measure([0, 1], [0, 1])
    return circuit


# Re-run simulation with the PSI- Bell state circuit (replace quantum_chsh_circuit with quantum_chsh_circuit_psi_minus in Step 3 code)
# --- Re-run Step 3 Simulation Code Here, but replace quantum_chsh_circuit with quantum_chsh_circuit_psi_minus ---

simulator = AerSimulator() # Using AerSimulator for potentially better performance
shots = 1024 # Number of game rounds to simulate

winning_probabilities_psi_minus = {} # Dictionary to store winning probabilities for each input

for x in [0, 1]:
    for y in [0, 1]:
        circuit = quantum_chsh_circuit_psi_minus(x, y) # <---- USE PSI- CIRCUIT HERE
        job = simulator.run(transpile(circuit, simulator), shots=shots) # Transpile for efficiency
        result = job.result()
        counts = result.get_counts(circuit)

        wins = 0
        for outcome, count in counts.items():
            alice_outcome = int(outcome[1]) # Alice's outcome is the second bit in the outcome string
            bob_outcome = int(outcome[0])   # Bob's outcome is the first bit in the outcome string

            # Determine win condition based on x and y
            if (x == 1 and y == 1): # Case XY=11: Win if A != B
                if alice_outcome != bob_outcome:
                    wins += count
            else: # Cases XY=00, 01, 10: Win if A == B
                if alice_outcome == bob_outcome:
                    wins += count

        win_probability = wins / shots
        winning_probabilities_psi_minus[(x, y)] = win_probability
        print(f"PSI- Bell State - Input (x={x}, y={y}): Winning Probability = {win_probability:.4f}")

print("\nWinning Probabilities for all Inputs (PSI- Bell State):", winning_probabilities_psi_minus)
average_winning_probability_psi_minus = np.mean(list(winning_probabilities_psi_minus.values()))
print(f"Average Winning Probability (Quantum Strategy - PSI- Bell State): {average_winning_probability_psi_minus:.4f}")

## Conclusion - Quantum Spookiness in Action!

Congratulations on completing the CHSH Game Lab!  In this lab, you have:

*   Implemented the quantum strategy for the CHSH game in Qiskit.
*   Simulated the game and verified the ~85% winning probability, exceeding the classical limit.
*   Witnessed a practical example of quantum advantage enabled by entanglement.
*   (Optionally) Explored classical strategies, angle variations, and different Bell states.

This lab demonstrates the fascinating and powerful nature of quantum entanglement.  It's not just "spooky action at a distance" - it's a real resource that can lead to tangible advantages in information processing and computation!

Keep exploring the world of quantum mechanics and quantum computing – there's much more to discover!

**Further Resources:**

*   Qiskit Textbook: [https://qiskit.org/textbook/](https://qiskit.org/textbook/)
*   Qiskit YouTube Channel: [https://www.youtube.com/qiskit](https://www.youtube.com/qiskit)

**Thank you for participating in the lab!**